# Compare performances

In [1]:
import os
import ast
import json

import pandas as pd

# Sklearn imports
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, auc, roc_curve

from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

<ipython-input-1-f0cb3b2954f1>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


## utils

In [2]:
def flatten(d):
    l = [ v for k,v in d.items() ]
    l = [item for sublist in l for item in sublist]
    l = list(map(int, l))
    return l

In [3]:
def binarize_fine(l):
    binarized_labels = []
    
    for i in l:
        if i > 1:
            i = 1
            binarized_labels.append( i )
        else:
            binarized_labels.append( i )
    
    return binarized_labels

In [4]:
def are_n_consecutive(numbers, n):
    n = 2
    if len(numbers) < n:
        return False
    for i in range(len(numbers) - n + 1):
        window = numbers[i:i+n]
        if max(window) - min(window) != n-1:
            continue
        if len(set(window)) == n:
            return True
    return False

In [5]:
def get_order():
    
    order = ['lf_Professional_Society.tsv',
    'lf_Biomedical_Occupation_or_Discipline.tsv',
    'lf_Idea_or_Concept.tsv',
    'lf_Classification.tsv',
    'lf_Biomedical_or_Dental_Material.tsv',
    'lf_Manufactured_Object.tsv',
    'lf_Functional_Concept.tsv',
    'lf_Temporal_Concept.tsv',
    'lf_Gene_or_Genome.tsv',
    'lf_Organic_Chemical.tsv',
    'lf_Biologically_Active_Substance.tsv',
    'lf_Medical_Device.tsv',
    'lf_Pharmacologic_Substance.tsv',
    'lf_Intellectual_Product.tsv',
    'lf_Diagnostic_Procedure.tsv',
    'lf_Laboratory_Procedure.tsv',
    'lf_Health_Care_Activity.tsv',
    'lf_Therapeutic_or_Preventive_Procedure.tsv',
    'lf_Finding.tsv']
    
    
    return order

## groundtruth

In [6]:
# load groundtruth for coarse matches

gt_file = '/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_bound/lf_Therapeutic_or_Preventive_Procedure.tsv'

# open file and load into a dataframe
groundtruth_df = pd.read_csv(gt_file, sep='\t', header=0)

In [7]:
# process the coarse and fine and add the pmid:gt to a ob dictionaries
gt_coarse = dict()
gt_fine = dict()

for identifier, x, y in zip(groundtruth_df['pmid'], groundtruth_df['i'], groundtruth_df['i_f']):
    x = ast.literal_eval(x)
    y = ast.literal_eval(y)
    
    gt_coarse[str(identifier)] = x
    gt_fine[str(identifier)] = y

In [8]:
print( 'Total number of PMIDs for groundtruth - coarse: ', len(gt_coarse) )
print( 'Total number of PMIDs for groundtruth - fine: ', len(gt_fine) )

Total number of PMIDs for groundtruth - coarse:  4802
Total number of PMIDs for groundtruth - fine:  4802


In [9]:
# flatten groundtruth

gt_coarse_flattened = flatten( gt_coarse )
gt_fine_flattened = flatten( gt_fine )

# binarize the fine groundtruth labels
gt_fine_flattened = binarize_fine(gt_fine_flattened)

## order-bound

In [10]:
in_order = get_order()

In [11]:
# load order bound matches for all dictionaries

ob_dir = '/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_bound'
ob_files = os.listdir(ob_dir)

In [12]:
ob_matches = dict()

for i in tqdm(in_order):
    print(i)
    file_path = f'{ob_dir}/{i}'
    
    filename = str(i).replace('lf_', '')
    filename = str(filename).replace('.tsv', '')
    
    # open file and load into a dataframe
    data_df = pd.read_csv(file_path, sep='\t', header=0)
    
    # process the labels and add the pmid:labels to a ob dictionary
    labs_all = []
    for x in data_df['labels']:
        x = ast.literal_eval(x)
        labs_all.append( x )
    ob_matches[filename] = dict( zip( data_df['pmid'],  pd.Series( labs_all )) )

  0%|          | 0/19 [00:00<?, ?it/s]

lf_Professional_Society.tsv


  5%|▌         | 1/19 [00:03<00:55,  3.09s/it]

lf_Biomedical_Occupation_or_Discipline.tsv


 11%|█         | 2/19 [00:06<00:51,  3.05s/it]

...

 89%|████████▉ | 17/19 [00:53<00:06,  3.30s/it]

lf_Therapeutic_or_Preventive_Procedure.tsv


 95%|█████████▍| 18/19 [00:55<00:02,  2.98s/it]

lf_Finding.tsv


100%|██████████| 19/19 [00:58<00:00,  3.10s/it]


In [13]:
print( 'Total number of PMIDs for orderbound files: ', len(ob_matches['Therapeutic_or_Preventive_Procedure']) )

Total number of PMIDs for orderbound files:  4802


In [14]:
assert len(gt_fine) == len( ob_matches['Therapeutic_or_Preventive_Procedure'] ) 

In [15]:
len(gt_fine)

4802

In [16]:
# calculate performance metrics for order-bound files

for k, v in ob_matches.items():
    
    v_flattened = flatten( v )
    
    # Classification report
    cr_order_bound_coarse = classification_report( gt_coarse_flattened, v_flattened, digits=4, output_dict=True)
    print(f'Confusion matrix for coarse-grained ground truth and {k} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_coarse['macro avg']['recall'], cr_order_bound_coarse['macro avg']['f1-score'], cr_order_bound_coarse['0']['precision'], cr_order_bound_coarse['0']['recall'], cr_order_bound_coarse['0']['f1-score'], cr_order_bound_coarse['1']['precision'], cr_order_bound_coarse['1']['recall'], cr_order_bound_coarse['1']['f1-score'])
    print( report )
    

    cr_order_bound_fine = classification_report( gt_fine_flattened, v_flattened, digits=4, output_dict=True)
    print(f'\n\nConfusion matrix for fine-grained ground truth and {k} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_fine['macro avg']['recall'], cr_order_bound_fine['macro avg']['f1-score'], cr_order_bound_fine['0']['precision'], cr_order_bound_fine['0']['recall'], cr_order_bound_fine['0']['f1-score'], cr_order_bound_fine['1']['precision'], cr_order_bound_fine['1']['recall'], cr_order_bound_fine['1']['f1-score'])
    print( report )
    
    print( '--------------------------------------------------------------------------' )

Confusion matrix for coarse-grained ground truth and Professional_Society matches
0.5000, 0.4746, 0.9033, 1.0000, 0.9492, 0.2000, 0.0000, 0.0000


Confusion matrix for fine-grained ground truth and Professional_Society matches
0.5000, 0.4821, 0.9307, 1.0000, 0.9641, 0.0000, 0.0000, 0.0000
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and Biomedical_Occupation_or_Discipline matches
0.5025, 0.4826, 0.9038, 0.9958, 0.9476, 0.1906, 0.0093, 0.0177


Confusion matrix for fine-grained ground truth and Biomedical_Occupation_or_Discipline matches
0.5038, 0.4922, 0.9312, 0.9958, 0.9624, 0.1730, 0.0118, 0.0221
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and Idea_or_Concept matches
0.4853, 0.4824, 0.9006, 0.9249, 0.9126, 0.0610, 0.0456, 0.0522


Confusion matrix for fine-grained ground truth and Idea_or_Concept matches
0.4855, 0.4858, 0.9287,

## order-free

In [60]:
# load order free matches with all dictionaries

picos = 'I' # ['P', 'I', 'O', 'S']
match_level = 'doc' # ['doc', 'sent', 'win_5', 'para']

In [18]:
order_free_dir = f'/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_free/{match_level}/{picos}'
order_free_files = os.listdir(order_free_dir)
print('Files: ', order_free_files)

Files:  ['Biomedical_or_Dental_Material.json', '.nfs0000000011200a4200000003', 'Classification.json', 'Intellectual_Product.json', 'Biologically_Active_Substance.json', 'Diagnostic_Procedure.json', 'Gene_or_Genome.json', 'Finding.json', 'Functional_Concept.json', 'Medical_Device.json', 'Organic_Chemical.json', 'Laboratory__Procedure.json', 'Manufactured_Object.json', 'train_ebm_intervention.json', 'Professional_Society.json', 'Pharmacologic_Substance.json', 'Therapeutic_or_Preventive_Procedure.json', 'train_ebm_intervention_syn.json', 'Biomedical_Occupation_or_Discipline.json', 'Health_Care_Activity.json', 'Idea_or_Concept.json', 'Temporal_Concept.json']


In [19]:
order_free_files.remove('.nfs0000000011200a4200000003')
#order_free_files.remove('Finding.json')

In [20]:
orf_loaded_files = dict()

for i in tqdm(order_free_files):
    
    filpath = f'{order_free_dir}/{i}'
    print('Loading file...', i)
    with open( filpath, 'r' ) as rf:
        orf_i = json.load(rf)
        orf_loaded_files[i] = orf_i

  0%|          | 0/21 [00:00<?, ?it/s]

Loading file... Biomedical_or_Dental_Material.json


  5%|▍         | 1/21 [00:02<00:40,  2.04s/it]

Loading file... Classification.json


 10%|▉         | 2/21 [00:03<00:30,  1.59s/it]

...

 90%|█████████ | 19/21 [09:01<01:11, 35.61s/it]

Loading file... Idea_or_Concept.json


 95%|█████████▌| 20/21 [09:01<00:25, 25.51s/it]

Loading file... Temporal_Concept.json


100%|██████████| 21/21 [09:02<00:00, 25.84s/it]


In [93]:
# set variables

set_partial_matches = False
set_actual_orf_matches = False

In [94]:
def get_orf(v, par, actual_orf):
    
    orfs = dict()
    
    for k_i, v_i in v.items():
        # print( k_i ) # example: name_15_3, name_9_9, name_12_8
        
        if 'Inters. (full)' in v_i and len(v_i['Inters. (full)']) > 0:
            
            full_inters = v_i['Inters. (full)']
            # full_inters.keys() = PMIDs
            # full_inters.values() = offsets, tokens

            for pmid, matches in full_inters.items():
                if len(matches['char offs.']) > 1:
                    if pmid not in orfs:
                        if actual_orf == True:
                            #if sorted( matches['word offs.'] ) != matches['word offs.']:
                            if are_n_consecutive(matches['word offs.'], 2) == False:
                                orfs[pmid] = [ ]
                                orfs[pmid].extend( matches['word offs.'] )      
                        elif actual_orf == False: 
                            orfs[pmid] = [ ]
                            orfs[pmid].extend( matches['word offs.'] )
                    else:
                        if actual_orf == True:
                            #if sorted( matches['word offs.'] ) != matches['word offs.']:
                            if are_n_consecutive(matches['word offs.'], 2) == False:
                                orfs[pmid].extend( matches['word offs.'] )
                        elif actual_orf == False:
                            orfs[pmid].extend( matches['word offs.'] )


        if 'Inters. (partial)' in v_i and len(v_i['Inters. (partial)']) > 0 and (par=='both' or par==True):
            
            par_inters = v_i['Inters. (partial)']
            
            for pmid, matches in par_inters.items():
                if len(matches['char offs.']) > 1:
                    if pmid not in orfs:
                        if actual_orf == True:
                            #if sorted( matches['word offs.'] ) != matches['word offs.']:
                            if are_n_consecutive(matches['word offs.'], 2) == False:
                                orfs[pmid] = [ ]
                                orfs[pmid].extend( matches['word offs.'] )   
                        elif actual_orf == False:
                            orfs[pmid] = [ ]
                            orfs[pmid].extend( matches['word offs.'] ) 
                    else:
                        if actual_orf == True:
                            #if sorted( matches['word offs.'] ) != matches['word offs.']:
                            if are_n_consecutive(matches['word offs.'], 2) == False:
                                orfs[pmid].extend( matches['word offs.'] )
                        elif actual_orf == False:
                            orfs[pmid].extend( matches['word offs.'] )
    
    return orfs

In [95]:
orfs_dict = dict()

for k,v in orf_loaded_files.items():
    
    new_k = str(k).replace('.json', '')

    orf_fetched = get_orf(v, par = set_partial_matches, actual_orf = set_actual_orf_matches)
    orfs_dict[new_k] = orf_fetched

In [96]:
# Order the loaded ORF matches in ascending order depending on the number of matches found

order_offset_dict = dict()

for k,v in orfs_dict.items():
    
    total_offsets = []
    for k_i, v_i in v.items():
        total_offsets.extend( v_i )
    
    order_offset_dict[k] = len(total_offsets)
    
    print( f'Offsets in {k}: ', len(total_offsets) )

Offsets in Biomedical_or_Dental_Material:  3258
Offsets in Classification:  8369
Offsets in Intellectual_Product:  86335
Offsets in Biologically_Active_Substance:  15494
Offsets in Diagnostic_Procedure:  33795
Offsets in Gene_or_Genome:  7226
Offsets in Finding:  298450
Offsets in Functional_Concept:  19792
Offsets in Medical_Device:  10628
Offsets in Organic_Chemical:  13818
Offsets in Laboratory__Procedure:  44098
Offsets in Manufactured_Object:  7624
Offsets in train_ebm_intervention:  2835
Offsets in Professional_Society:  8
Offsets in Pharmacologic_Substance:  37882
Offsets in Therapeutic_or_Preventive_Procedure:  145151
Offsets in train_ebm_intervention_syn:  25540
Offsets in Biomedical_Occupation_or_Discipline:  4425
Offsets in Health_Care_Activity:  60790
Offsets in Idea_or_Concept:  13321
Offsets in Temporal_Concept:  70258


In [97]:
# sort dictionary
order_offset_sorteddict = dict(sorted(order_offset_dict.items(), key=lambda x: x[1]))

In [98]:
print( 'Total number of dictionaries loaded: ', len(order_offset_sorteddict) )

Total number of dictionaries loaded:  21


In [99]:
# temporary - to remove
del order_offset_sorteddict['train_ebm_intervention_syn']
del order_offset_sorteddict['train_ebm_intervention']

orfs_dict["Laboratory_Procedure"] = orfs_dict.pop("Laboratory__Procedure")
order_offset_sorteddict["Laboratory_Procedure"] = order_offset_sorteddict.pop("Laboratory__Procedure")

In [100]:
assert len( order_offset_sorteddict.keys() ) == len( ob_matches.keys() )
assert sorted( order_offset_sorteddict.keys() ) == sorted( ob_matches.keys() )

In [101]:
list( order_offset_sorteddict.keys() )

['Professional_Society',
 'Biomedical_or_Dental_Material',
 'Biomedical_Occupation_or_Discipline',
 'Gene_or_Genome',
 'Manufactured_Object',
 'Classification',
 'Medical_Device',
 'Idea_or_Concept',
 'Organic_Chemical',
 'Biologically_Active_Substance',
 'Functional_Concept',
 'Diagnostic_Procedure',
 'Pharmacologic_Substance',
 'Health_Care_Activity',
 'Temporal_Concept',
 'Intellectual_Product',
 'Therapeutic_or_Preventive_Procedure',
 'Finding',
 'Laboratory_Procedure']

In [102]:
def merge_preds_ordered( ob_preds, orf_offsets ):
    
    merged_predictions = dict()
    
    for k, v in ob_preds.items(): # ob preds are the old preds
        
        k = str(k)

        if k in orf_offsets:
            matching_offsets = orf_offsets[k]
            old_preds = list( map( int, v ))
            new_preds = list( map( int, v ))
                             
            # add new offsets to the new_preds
            for indice in matching_offsets:
                new_preds[indice] = 1

            merged_predictions[ k ] = new_preds
        else:
            merged_predictions[ k ] = list( map( int, v ) )
    
    return merged_predictions

In [ ]:
# full matches - Order free matches
for count, i in enumerate(in_order):
    
    to_fetch = str(i).replace('lf_', '').replace('.tsv', '')
    
    base_preds = ob_matches[to_fetch]
    base_preds_flattened =  flatten( base_preds )
    
    updated_of_preds = merge_preds_ordered( base_preds, orfs_dict[to_fetch] )
    updated_preds_flattened =  flatten( updated_of_preds )
    
    # Classification report
    cr_order_bound_coarse = classification_report( gt_coarse_flattened, updated_preds_flattened, digits=4, output_dict=True)
    print(f'Confusion matrix for coarse-grained ground truth and {i} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_coarse['macro avg']['recall'], cr_order_bound_coarse['macro avg']['f1-score'], cr_order_bound_coarse['0']['precision'], cr_order_bound_coarse['0']['recall'], cr_order_bound_coarse['0']['f1-score'], cr_order_bound_coarse['1']['precision'], cr_order_bound_coarse['1']['recall'], cr_order_bound_coarse['1']['f1-score'])
    print( report )

    cr_order_bound_fine = classification_report( gt_fine_flattened, updated_preds_flattened, digits=4, output_dict=True)
    print(f'\n\nConfusion matrix for fine-grained ground truth and {i} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_fine['macro avg']['recall'], cr_order_bound_fine['macro avg']['f1-score'], cr_order_bound_fine['0']['precision'], cr_order_bound_fine['0']['recall'], cr_order_bound_fine['0']['f1-score'], cr_order_bound_fine['1']['precision'], cr_order_bound_fine['1']['recall'], cr_order_bound_fine['1']['f1-score'])
    print( report )
    
    print( '--------------------------------------------------------------------------' )

Confusion matrix for coarse-grained ground truth and lf_Professional_Society.tsv matches
0.5000, 0.4746, 0.9033, 1.0000, 0.9492, 0.2105, 0.0000, 0.0001


Confusion matrix for fine-grained ground truth and lf_Professional_Society.tsv matches
0.5000, 0.4821, 0.9307, 1.0000, 0.9641, 0.0000, 0.0000, 0.0000
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and lf_Biomedical_Occupation_or_Discipline.tsv matches
0.5027, 0.4839, 0.9038, 0.9944, 0.9470, 0.1749, 0.0110, 0.0207


Confusion matrix for fine-grained ground truth and lf_Biomedical_Occupation_or_Discipline.tsv matches
0.5042, 0.4936, 0.9313, 0.9945, 0.9618, 0.1573, 0.0138, 0.0254
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and lf_Idea_or_Concept.tsv matches
0.4861, 0.4841, 0.9007, 0.9203, 0.9104, 0.0651, 0.0519, 0.0577


Confusion matrix for fine-grained ground truth and lf_Idea_or_C

In [79]:
# partial matches - order free matches

for count, i in enumerate(in_order):
    
    to_fetch = str(i).replace('lf_', '').replace('.tsv', '')
    
    base_preds = ob_matches[to_fetch]
    base_preds_flattened =  flatten( base_preds )
    
    updated_of_preds = merge_preds_ordered( base_preds, orfs_dict[to_fetch] )
    updated_preds_flattened =  flatten( updated_of_preds )
    
    # Classification report
    cr_order_bound_coarse = classification_report( gt_coarse_flattened, updated_preds_flattened, digits=4, output_dict=True)
    print(f'Confusion matrix for coarse-grained ground truth and {i} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_coarse['macro avg']['recall'], cr_order_bound_coarse['macro avg']['f1-score'], cr_order_bound_coarse['0']['precision'], cr_order_bound_coarse['0']['recall'], cr_order_bound_coarse['0']['f1-score'], cr_order_bound_coarse['1']['precision'], cr_order_bound_coarse['1']['recall'], cr_order_bound_coarse['1']['f1-score'])
    print( report )

    cr_order_bound_fine = classification_report( gt_fine_flattened, updated_preds_flattened, digits=4, output_dict=True)
    print(f'\n\nConfusion matrix for fine-grained ground truth and {i} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_fine['macro avg']['recall'], cr_order_bound_fine['macro avg']['f1-score'], cr_order_bound_fine['0']['precision'], cr_order_bound_fine['0']['recall'], cr_order_bound_fine['0']['f1-score'], cr_order_bound_fine['1']['precision'], cr_order_bound_fine['1']['recall'], cr_order_bound_fine['1']['f1-score'])
    print( report )
    
    print( '--------------------------------------------------------------------------' )

Confusion matrix for coarse-grained ground truth and lf_Professional_Society.tsv matches
0.5000, 0.4750, 0.9033, 0.9993, 0.9489, 0.0862, 0.0006, 0.0012


Confusion matrix for fine-grained ground truth and lf_Professional_Society.tsv matches
0.5000, 0.4825, 0.9307, 0.9993, 0.9638, 0.0632, 0.0006, 0.0012
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and lf_Biomedical_Occupation_or_Discipline.tsv matches
0.5035, 0.4876, 0.9040, 0.9904, 0.9452, 0.1560, 0.0166, 0.0300


Confusion matrix for fine-grained ground truth and lf_Biomedical_Occupation_or_Discipline.tsv matches
0.5055, 0.4978, 0.9314, 0.9905, 0.9600, 0.1376, 0.0205, 0.0356
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and lf_Idea_or_Concept.tsv matches
0.4844, 0.4847, 0.9003, 0.8957, 0.8980, 0.0697, 0.0730, 0.0713


Confusion matrix for fine-grained ground truth and lf_Idea_or_C

## actual order-free

In [ ]:
# load (actualin in in in ) order free matches with all dictionaries

In [92]:
# partial matches - ACTUAL order free matches

for count, i in enumerate(in_order):
    
    to_fetch = str(i).replace('lf_', '').replace('.tsv', '')
    
    base_preds = ob_matches[to_fetch]
    base_preds_flattened =  flatten( base_preds )
    
    updated_of_preds = merge_preds_ordered( base_preds, orfs_dict[to_fetch] )
    updated_preds_flattened =  flatten( updated_of_preds )
    
    # Classification report
    cr_order_bound_coarse = classification_report( gt_coarse_flattened, updated_preds_flattened, digits=4, output_dict=True)
    print(f'Confusion matrix for coarse-grained ground truth and {i} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_coarse['macro avg']['recall'], cr_order_bound_coarse['macro avg']['f1-score'], cr_order_bound_coarse['0']['precision'], cr_order_bound_coarse['0']['recall'], cr_order_bound_coarse['0']['f1-score'], cr_order_bound_coarse['1']['precision'], cr_order_bound_coarse['1']['recall'], cr_order_bound_coarse['1']['f1-score'])
    print( report )

    cr_order_bound_fine = classification_report( gt_fine_flattened, updated_preds_flattened, digits=4, output_dict=True)
    print(f'\n\nConfusion matrix for fine-grained ground truth and {i} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_fine['macro avg']['recall'], cr_order_bound_fine['macro avg']['f1-score'], cr_order_bound_fine['0']['precision'], cr_order_bound_fine['0']['recall'], cr_order_bound_fine['0']['f1-score'], cr_order_bound_fine['1']['precision'], cr_order_bound_fine['1']['recall'], cr_order_bound_fine['1']['f1-score'])
    print( report )
    
    print( '--------------------------------------------------------------------------' )

Confusion matrix for coarse-grained ground truth and lf_Professional_Society.tsv matches
0.5000, 0.4751, 0.9033, 0.9994, 0.9489, 0.0948, 0.0006, 0.0012


Confusion matrix for fine-grained ground truth and lf_Professional_Society.tsv matches
0.5000, 0.4825, 0.9307, 0.9994, 0.9639, 0.0701, 0.0006, 0.0012
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and lf_Biomedical_Occupation_or_Discipline.tsv matches
0.5035, 0.4872, 0.9040, 0.9910, 0.9455, 0.1592, 0.0160, 0.0290


Confusion matrix for fine-grained ground truth and lf_Biomedical_Occupation_or_Discipline.tsv matches
0.5054, 0.4975, 0.9314, 0.9911, 0.9603, 0.1411, 0.0197, 0.0346
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and lf_Idea_or_Concept.tsv matches
0.4847, 0.4849, 0.9004, 0.8971, 0.8987, 0.0700, 0.0724, 0.0711


Confusion matrix for fine-grained ground truth and lf_Idea_or_C